# 7 - Tick Priips (C#)

This sample reads from a .csv file with a list of trades and returns a file with arrival cost.

### Setup and import packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.32.*"
#r "nuget: CsvHelper"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.TickAnalytics.V1;
using TickTradesService = Systemathics.Apis.Services.Tick.V1.TickTradesService;
using TickTradesRequest = Systemathics.Apis.Services.Tick.V1.TickTradesRequest;
using TickTradesResponse = Systemathics.Apis.Services.Tick.V1.TickTradesResponse;
using Trade = Systemathics.Apis.Type.Shared.V1.Trade;

using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
// Import/export csv files 
using CsvHelper;
using System.IO;
using System.Threading;
using System.Globalization;

### Retrieve the token

In [ ]:
var headers = TokenHelpers.GetTokenAsMetaData();

### Load inputs

In [ ]:
//var inputFilename = "input/input20230307_564_787.csv";
var inputFilename = "input/priips_sample_input.csv";
var outputFilename = "output/priips_sample_output.csv";
var latency = new Duration { Seconds = 60 * 60 };
var maxProcesses = 10000; // set a maximum number of rows to process

In [ ]:
// Input trade structure to load from the csv file
public class InputTrade
{
    public string GUID { get; set; }
    public string ISIN { get; set; } 
    public string STOCK_EXCHANGE { get; set; } 
    public string INSTR_DESC { get; set; } 
    public string TRA_TYPE { get; set; }
    public string EXECUTION_PRICE { get; set; }
    public string ORDER_TIMESTAMP { get; set; }
    public string EXECUTION_TIMESTAMP { get; set; }
    public string Source { get; set; }
    public string Symbol { get; set; }
}

In [ ]:
// Load the trades from the csv input file
var records = new List<InputTrade>();
using (var reader = new StreamReader(inputFilename))
using (var csv = new CsvReader(reader, CultureInfo.InvariantCulture))
{
    records = csv.GetRecords<InputTrade>().ToList();
}

In [ ]:
// display records content
display(records)

### Process Request
Read the input file, process the request for each row, and write results in the output file on the fly

In [ ]:
#!time
var d = new DirectoryInfo("output"); if (!d.Exists) { d.Create(); }
var field = PriipsPrice.Trade;
using (var fs = new StreamWriter(File.Create(outputFilename))) 
{
    // Write headers in output file
    fs.WriteLine($"Guid,Symbol,Source,Exchange,Isin,Instrument_descriptor,Type,ArrivalTime,ArrivalPrice,ArrivalCost,OpenTime,CloseTime,OrderTime,ExecutionTime,Duration (s),Latency (s),ExecutionPrice,Count,OpenPrice,HighPrice,LowPrice,ClosePrice,Vwap,Volume");
    
    // prepare channel
    var channel = ChannelHelpers.GetChannel();
    var tickPriipsService = new TickPriipsService.TickPriipsServiceClient(channel);
    
    for (int i =0; i < Math.Min(maxProcesses,records.Count) ; i++) 
    {
        var record = records[i];
        var mappedExchange = record.Source;
        var mappedTicker = record.Symbol;
        var ot = DateTime.Parse(record.ORDER_TIMESTAMP).ToUniversalTime();
        var et = DateTime.Parse(record.EXECUTION_TIMESTAMP).ToUniversalTime();
        var tickPriipsRequest = new TickPriipsRequest 
                                                { 
                                                    Identifier = new Identifier { Exchange = mappedExchange , Ticker = mappedTicker},
                                                    OrderTime = Timestamp.FromDateTime(ot),
                                                    ExecutionTime = Timestamp.FromDateTime(et),
                                                    ExecutionPrice = Double.Parse(record.EXECUTION_PRICE, System.Globalization.CultureInfo.InvariantCulture),
                                                    Field = field,
                                                    Latency = latency
                                                };
        
        
        // Fields from input .csv file 
        var guid = record.GUID;
        var isin = record.ISIN;
        var stockExchange = record.STOCK_EXCHANGE;
        var instrDescriptor = record.INSTR_DESC;
        var tradeType = record.TRA_TYPE;

        
        try 
        {
            var tickPriipsCall = tickPriipsService.TickPriips(tickPriipsRequest, headers);
            await foreach (var current in tickPriipsCall.ResponseStream.ReadAllAsync())
            {
                // Fields from Priips service response
                var arrivalTime = current.ArrivalTime;
                var arrivalPrice = current.ArrivalPrice;
                var arrivalCost = current.ArrivalCost;
                var openTime = current.OpenTime;
                var closeTime = current.CloseTime;
                var orderTime = current.OrderTime;
                var executionTime = current.ExecutionTime;
                var duration = current.Duration.Seconds;
                var executionPrice = current.ExecutionPrice;
                var count = current.Count;
                var openPrice = current.OpenPrice;
                var highPrice = current.HighPrice;
                var lowPrice = current.LowPrice;
                var closePrice = current.ClosePrice;
                var vwap = current.Vwap;
                var volume = current.Volume;
                
                // arrivalPeriod
                var arrivalPeriod = orderTime - arrivalTime;
                var arrivalPeriodStr = $"{arrivalPeriod}";
                arrivalPeriodStr = arrivalPeriodStr.Replace("s", string.Empty);  
                
                // Export
                fs.WriteLine($"{guid},{mappedTicker},{mappedExchange},{stockExchange},{isin},{instrDescriptor},{tradeType},{arrivalTime},{arrivalPrice},{arrivalCost},{openTime},{closeTime},{orderTime},{executionTime},{duration},{latency},{executionPrice},{count},{openPrice},{highPrice},{lowPrice},{closePrice},{vwap},{volume}");

            }   
        } 
        catch 
        {
            var inputOrderTime = $"{record.ORDER_TIMESTAMP}".Replace(" ", "T").Replace(".000", "Z");
            var inputExecutionTime = $"{record.EXECUTION_TIMESTAMP}".Replace(" ", "T").Replace(".000", "Z");
            fs.WriteLine($"{guid},{mappedTicker},{mappedExchange},{stockExchange},{isin},{instrDescriptor},{tradeType},,,,,,{inputOrderTime},{inputExecutionTime},,{latency},,,,,,,,");

        }
        
 
    }
    
}
Console.WriteLine($"Done exporting data ({records.Count} records)");